# Introduction

The prices of HDB flats in Singapore is of interest to most Singaporeans as it is the most affordable and accessible form of housing for young adults, who wish to move out of their parent's. There are 2 ways to go about obtaining one: ballot for a new flat from HDB (opportunity to buy at a subsidised rate!), or get a resale flat on the open market (some subsidies, but ho-hum). There are no regulations to fix prices (ceiling or floor) for resale flats, so transaction prices are entirely up to buyers and sellers. They may value a flat based on objective characteristics (area, floor level) or subjective features (well-maintained, good location). Having been in a position shopping for a flat, the question of 'Is this a fair price to pay for the flat?' came up repeatedly and I resorted to checking the recent resale transactions around the area (a manual version of k-nearest neighbours, if you will) to decide the fair value of a flat that I am interested in. 

The Singapore government launched a portal, Data.gov.sg, in an effort to make public data more accessible and I found that HDB resale transactions going back all the way to 1990 are available in a csv format right [here]( https://data.gov.sg/dataset/resale-flat-prices?resource_id=83b2fc37-ce8c-4df4-968b-370fd818138b).

In this workbook, I will detail my steps to creating a linear regression model using the public information available in these csv files.

# Getting files and combining them

Obviously the first task will be to get the files by downloading it off the site, data.gov.sg does provide an API for queries but since we just after the csv files available at the click of a button, why bother hacking through all the API documentations when you can do it the easy way? So download all of them in 1 zip file and unzip it!

Now you got them, let's see what we're working with here!

In [1]:
# Getting all the necessary packages
import numpy as np
import pandas as pd

In [2]:
df1990 = pd.read_csv('resale_data/resale-flat-prices-based-on-approval-date-1990-1999.csv')
df1990.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price
0,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,IMPROVED,1977,9000
1,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,04 TO 06,31.0,IMPROVED,1977,6000
2,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,IMPROVED,1977,8000
3,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,07 TO 09,31.0,IMPROVED,1977,6000
4,1990-01,ANG MO KIO,3 ROOM,216,ANG MO KIO AVE 1,04 TO 06,73.0,NEW GENERATION,1976,47200


In [3]:
df1990.isna().sum()

month                  0
town                   0
flat_type              0
block                  0
street_name            0
storey_range           0
floor_area_sqm         0
flat_model             0
lease_commence_date    0
resale_price           0
dtype: int64

No missing data, very nice! Ok now you've done it for 1, and the data looks good and clean, let's go do it for the other 4!

In [4]:
df2000 = pd.read_csv('resale_data/resale-flat-prices-based-on-approval-date-2000-feb-2012.csv')
df2012 = pd.read_csv('resale_data/resale-flat-prices-based-on-registration-date-from-mar-2012-to-dec-2014.csv')
df2015 = pd.read_csv('resale_data/resale-flat-prices-based-on-registration-date-from-jan-2015-to-dec-2016.csv')
df2017 = pd.read_csv('resale_data/resale-flat-prices-based-on-registration-date-from-jan-2017-onwards.csv')
df2017.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61 years 04 months,232000.0
1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,60 years 07 months,250000.0
2,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,262000.0
3,2017-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1980,62 years 01 month,265000.0
4,2017-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,265000.0


### Looks alright, but note that there's an extra column: 'remaining_lease'. Now this is going to be a problem potentially...

solution: we'll add the remaining_lease column to the other years as we have the date the lease commences, assuming a 99 year lease (most HDBs are with some exceptional 60 year ones). 
``` remaining_lease = lease commence + 99 - year of transaction```

Lets find out which years we need to impute the 'remaining_lease' columns

In [5]:
df_list = [df1990, df2000, df2012, df2015, df2017]
for df in df_list:
    if 'remaining_lease' not in df.columns:
        print(df.iloc[1,:])

month                           1990-01
town                         ANG MO KIO
flat_type                        1 ROOM
block                               309
street_name            ANG MO KIO AVE 1
storey_range                   04 TO 06
floor_area_sqm                       31
flat_model                     IMPROVED
lease_commence_date                1977
resale_price                       6000
Name: 1, dtype: object
month                           2000-01
town                         ANG MO KIO
flat_type                        3 ROOM
block                               174
street_name            ANG MO KIO AVE 4
storey_range                   04 TO 06
floor_area_sqm                       61
flat_model                     Improved
lease_commence_date                1986
resale_price                     144000
Name: 1, dtype: object
month                           2012-03
town                         ANG MO KIO
flat_type                        2 ROOM
block                             

Here we see, only df1990, 2000 and 2012 does not have the 'remaining_lease' column, let's create one for them. 

In [6]:
for df in df_list[0:3]:
    df['year'] = df['month'].str[:4] # transaction dates are saved as a string, with the years in the first 4 columns
    df['year'] = df['year'].astype(int)
    df['remaining_lease'] = df['lease_commence_date'] + 99 - df['year']
df2000.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,year,remaining_lease
0,2000-01,ANG MO KIO,3 ROOM,170,ANG MO KIO AVE 4,07 TO 09,69.0,Improved,1986,147000.0,2000,85
1,2000-01,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,04 TO 06,61.0,Improved,1986,144000.0,2000,85
2,2000-01,ANG MO KIO,3 ROOM,216,ANG MO KIO AVE 1,07 TO 09,73.0,New Generation,1976,159000.0,2000,75
3,2000-01,ANG MO KIO,3 ROOM,215,ANG MO KIO AVE 1,07 TO 09,73.0,New Generation,1976,167000.0,2000,75
4,2000-01,ANG MO KIO,3 ROOM,218,ANG MO KIO AVE 1,07 TO 09,67.0,New Generation,1976,163000.0,2000,75


Everythign looks fine here, we're going to drop the 'years' column to avoid future problems when we combine the dataframes!

In [7]:
for df in df_list[0:3]:
    df.drop(columns=['year'],inplace = True)
df2000.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease
0,2000-01,ANG MO KIO,3 ROOM,170,ANG MO KIO AVE 4,07 TO 09,69.0,Improved,1986,147000.0,85
1,2000-01,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,04 TO 06,61.0,Improved,1986,144000.0,85
2,2000-01,ANG MO KIO,3 ROOM,216,ANG MO KIO AVE 1,07 TO 09,73.0,New Generation,1976,159000.0,75
3,2000-01,ANG MO KIO,3 ROOM,215,ANG MO KIO AVE 1,07 TO 09,73.0,New Generation,1976,167000.0,75
4,2000-01,ANG MO KIO,3 ROOM,218,ANG MO KIO AVE 1,07 TO 09,67.0,New Generation,1976,163000.0,75


In [8]:
df2017.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61 years 04 months,232000.0
1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,60 years 07 months,250000.0
2,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,262000.0
3,2017-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1980,62 years 01 month,265000.0
4,2017-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,265000.0


For transactions from 2017, remaining leases are recorded as years and months, we'll just drop the months by slicing the first 2 chars (years) off the string since it's easier to do and probably not much data loss here.

In [9]:
df2017['remaining_lease'] = df2017['remaining_lease'].str[:2]
df2017['remaining_lease'] = df2017['remaining_lease'].astype(int)

In [10]:
# Just to check that there aren't any weird values sliced off!
df2017['remaining_lease'].unique()

array([61, 60, 62, 63, 58, 59, 93, 94, 78, 70, 67, 68, 56, 66, 65, 69, 64,
       83, 87, 71, 92, 54, 82, 75, 55, 74, 72, 80, 86, 79, 53, 57, 52, 51,
       95, 90, 81, 85, 84, 77, 88, 49, 50, 89, 73, 48, 76, 91, 96, 47, 46,
       45, 97])

# Now we're ready to combine the data!

In [11]:
transactions = pd.concat(df_list)
transactions.tail(10) #check random rows of the data to see if data has been combined successfully!

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease
80364,2020-09,YISHUN,5 ROOM,504D,YISHUN ST 51,13 TO 15,113.0,Improved,2016,630000.0,94
80365,2020-09,YISHUN,5 ROOM,504B,YISHUN ST 51,07 TO 09,112.0,Improved,2016,538000.0,94
80366,2020-09,YISHUN,5 ROOM,505B,YISHUN ST 51,04 TO 06,113.0,Improved,2016,540000.0,94
80367,2020-09,YISHUN,5 ROOM,637,YISHUN ST 61,10 TO 12,121.0,Improved,1987,459988.0,66
80368,2020-09,YISHUN,5 ROOM,626,YISHUN ST 61,04 TO 06,121.0,Improved,1987,460000.0,65
80369,2020-09,YISHUN,5 ROOM,716,YISHUN ST 71,07 TO 09,131.0,Improved,1987,440000.0,66
80370,2020-09,YISHUN,5 ROOM,760,YISHUN ST 72,07 TO 09,122.0,Improved,1987,458000.0,65
80371,2020-09,YISHUN,5 ROOM,835,YISHUN ST 81,04 TO 06,122.0,Improved,1987,490000.0,66
80372,2020-09,YISHUN,EXECUTIVE,791,YISHUN AVE 2,04 TO 06,146.0,Maisonette,1987,558000.0,66
80373,2020-09,YISHUN,EXECUTIVE,387,YISHUN RING RD,04 TO 06,146.0,Maisonette,1988,555000.0,66


In [12]:
transactions.isna().sum() #check if any null data exists, indicates something went wrong in the concatenation!

month                  0
town                   0
flat_type              0
block                  0
street_name            0
storey_range           0
floor_area_sqm         0
flat_model             0
lease_commence_date    0
resale_price           0
remaining_lease        0
dtype: int64

In [13]:
transactions.shape

(826581, 11)

We have 826581 transactions and 11 features, from which to make a simple regression model!

**A few considerations before starting to fit a regression model:**

* Resale prices have grown tremendously from 1990s (start of data set), how will I account for the price changes due to inflation? 
* Location is one of the obvious factor for prices, we have 2 features, street name and block that code for this property, but working with block and street names are always tricky and users might key in the same value differently e.g. Yishun Street v.s. Yishun St.
* Are there differences in data entry format that could same data to be interpreted differently?
* Storey ranges: traditionally, the higher the floor, the better, how should I best represent them?

Ideas to address issues:
* Transaction dates are included, if this is included as a feature, inflation should be addressed by this feature
* I could concatenate the block and street names to form a complete address, but in order to make it easy to use, I'll try converting it to a postal code (long side story for another post)
* Explore the data and standardise/replace data entry format!
* Data provided is already binned, let's convert them into ordinal features

In [14]:
transactions['flat_type'].unique()

array(['1 ROOM', '3 ROOM', '4 ROOM', '5 ROOM', '2 ROOM', 'EXECUTIVE',
       'MULTI GENERATION', 'MULTI-GENERATION'], dtype=object)

Looks like a simple replacement of 'MULTI-GENERATION' with ''MULTI GENERATION' will do the trick!

In [15]:
flat_type_dup = {'MULTI-GENERATION' : 'MULTI GENERATION'}
transactions.replace({'flat_type':flat_type_dup}, inplace = True)
transactions['flat_type'].unique() #Done!

array(['1 ROOM', '3 ROOM', '4 ROOM', '5 ROOM', '2 ROOM', 'EXECUTIVE',
       'MULTI GENERATION'], dtype=object)

In [16]:
transactions['flat_model'].unique()

array(['IMPROVED', 'NEW GENERATION', 'MODEL A', 'STANDARD', 'SIMPLIFIED',
       'MODEL A-MAISONETTE', 'APARTMENT', 'MAISONETTE', 'TERRACE',
       '2-ROOM', 'IMPROVED-MAISONETTE', 'MULTI GENERATION',
       'PREMIUM APARTMENT', 'Improved', 'New Generation', 'Model A',
       'Standard', 'Apartment', 'Simplified', 'Model A-Maisonette',
       'Maisonette', 'Multi Generation', 'Adjoined flat',
       'Premium Apartment', 'Terrace', 'Improved-Maisonette',
       'Premium Maisonette', '2-room', 'Model A2', 'DBSS', 'Type S1',
       'Type S2', 'Premium Apartment Loft'], dtype=object)

Seems like it's a classic upper/lower case mismatch during data entry! Let's just change it all to upper case!

In [17]:
transactions['flat_model'] = transactions['flat_model'].str.upper()
transactions['flat_model'].unique()

array(['IMPROVED', 'NEW GENERATION', 'MODEL A', 'STANDARD', 'SIMPLIFIED',
       'MODEL A-MAISONETTE', 'APARTMENT', 'MAISONETTE', 'TERRACE',
       '2-ROOM', 'IMPROVED-MAISONETTE', 'MULTI GENERATION',
       'PREMIUM APARTMENT', 'ADJOINED FLAT', 'PREMIUM MAISONETTE',
       'MODEL A2', 'DBSS', 'TYPE S1', 'TYPE S2', 'PREMIUM APARTMENT LOFT'],
      dtype=object)

## Duplicates removed!

In [18]:
storey_ranges = transactions['storey_range'].unique()
storey_ranges.sort()
storey_ranges

array(['01 TO 03', '01 TO 05', '04 TO 06', '06 TO 10', '07 TO 09',
       '10 TO 12', '11 TO 15', '13 TO 15', '16 TO 18', '16 TO 20',
       '19 TO 21', '21 TO 25', '22 TO 24', '25 TO 27', '26 TO 30',
       '28 TO 30', '31 TO 33', '31 TO 35', '34 TO 36', '36 TO 40',
       '37 TO 39', '40 TO 42', '43 TO 45', '46 TO 48', '49 TO 51'],
      dtype=object)

Problem: These are in overlapping bins! 01-03 and 01-05?! Also it goes all the way to 51? I have a feeling distributions of flats above 15 are going to be tiny?

In [19]:
transactions['storey_range'].value_counts()

04 TO 06    209705
07 TO 09    189109
01 TO 03    168985
10 TO 12    160250
13 TO 15     52212
16 TO 18     19449
19 TO 21      9422
22 TO 24      6053
01 TO 05      2700
25 TO 27      2544
06 TO 10      2474
11 TO 15      1259
28 TO 30      1049
34 TO 36       267
31 TO 33       265
16 TO 20       265
37 TO 39       255
40 TO 42       132
21 TO 25        92
26 TO 30        39
46 TO 48        21
43 TO 45        16
49 TO 51         9
36 TO 40         7
31 TO 35         2
Name: storey_range, dtype: int64

Maybe we should bin levels above 31 together? Side note: you can visualize this with a histogram too with .hist()
Back to our storey ranges, we have 2 tasks:

* remove the overlapping ones by joining them
* bin all the levels 31 and above

There's probably a smart way of doing this by algo, but there aren't that many groups, so I'm going to do this manually.

In [20]:
transactions.loc[transactions['storey_range']=='01 TO 03','storey_range'] = '01 TO 05'
transactions.loc[transactions['storey_range']=='04 TO 06','storey_range'] = '01 TO 05'
transactions.loc[transactions['storey_range']=='07 TO 09','storey_range'] = '06 TO 10'
transactions.loc[transactions['storey_range']=='10 TO 12','storey_range'] = '11 TO 15'
transactions.loc[transactions['storey_range']=='13 TO 15','storey_range'] = '11 TO 15'
transactions.loc[transactions['storey_range']=='16 TO 18','storey_range'] = '16 TO 20'
transactions.loc[transactions['storey_range']=='19 TO 21','storey_range'] = '16 TO 20'
transactions.loc[transactions['storey_range']=='22 TO 24','storey_range'] = '21 TO 25'
transactions.loc[transactions['storey_range']=='25 TO 27','storey_range'] = '26 TO 30'
transactions.loc[transactions['storey_range']=='28 TO 30','storey_range'] = '26 TO 30'

In [21]:
storey_ranges = transactions['storey_range'].unique()
storey_ranges.sort()
storey_ranges

array(['01 TO 05', '06 TO 10', '11 TO 15', '16 TO 20', '21 TO 25',
       '26 TO 30', '31 TO 33', '31 TO 35', '34 TO 36', '36 TO 40',
       '37 TO 39', '40 TO 42', '43 TO 45', '46 TO 48', '49 TO 51'],
      dtype=object)

In [22]:
idx = list(storey_ranges).index('31 TO 33')
idx

6

In [23]:
# hacky way of binning levels above 31
for i in range(idx,15):
    storey = storey_ranges[i]
    transactions.loc[transactions['storey_range'] == storey, 'storey_range'] = '31 TO 51'
transactions['storey_range'].unique()
# Looks good!

array(['11 TO 15', '01 TO 05', '06 TO 10', '16 TO 20', '26 TO 30',
       '21 TO 25', '31 TO 51'], dtype=object)

Now that it's cleaned, let's give it make it an ordinal feature, as we know that higher level units are more desirable than lower ones (for a variety of reasons), so higher > lower!

In [24]:
storey_ranges = transactions['storey_range'].unique()
storey_ranges.sort()
storey_converter = {v:k for k,v in enumerate(storey_ranges)}
storey_converter

{'01 TO 05': 0,
 '06 TO 10': 1,
 '11 TO 15': 2,
 '16 TO 20': 3,
 '21 TO 25': 4,
 '26 TO 30': 5,
 '31 TO 51': 6}

In [25]:
# Replacing the data
transactions.replace({"storey_range":storey_converter},inplace=True)
transactions.head(5)

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease
0,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,2,31.0,IMPROVED,1977,9000.0,86
1,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,0,31.0,IMPROVED,1977,6000.0,86
2,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,2,31.0,IMPROVED,1977,8000.0,86
3,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,1,31.0,IMPROVED,1977,6000.0,86
4,1990-01,ANG MO KIO,3 ROOM,216,ANG MO KIO AVE 1,0,73.0,NEW GENERATION,1976,47200.0,85


In [26]:
transactions['storey_range'].value_counts()

0    381390
2    213721
1    191583
3     29136
4      6145
5      3632
6       974
Name: storey_range, dtype: int64

## Now to address the fact that the transactions are over 30 years and the date of transaction undeniably has an impact on the resale price!

However, SKlearn does not accept date/time formats as features so we are going to have to convert it to an ordinal number from day 0, day 0 being the 1st transaction date, January 1990

In [27]:
transactions.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease
0,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,2,31.0,IMPROVED,1977,9000.0,86
1,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,0,31.0,IMPROVED,1977,6000.0,86
2,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,2,31.0,IMPROVED,1977,8000.0,86
3,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,1,31.0,IMPROVED,1977,6000.0,86
4,1990-01,ANG MO KIO,3 ROOM,216,ANG MO KIO AVE 1,0,73.0,NEW GENERATION,1976,47200.0,85


In [28]:
transactions['month'] = pd.to_datetime(transactions['month'])
firstdate = transactions.iloc[0,0]
transactions['tx_day'] = transactions['month'] - firstdate
transactions.sample(5)

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease,tx_day
261523,1999-06-01,TAMPINES,5 ROOM,303,TAMPINES ST 32,0,126.0,IMPROVED,1993,370000.0,93,3438 days
37632,2018-10-01,JURONG WEST,4 ROOM,447,JURONG WEST ST 42,0,103.0,MODEL A,1984,300000.0,64,10500 days
277539,1999-10-01,BEDOK,3 ROOM,116,BEDOK NTH RD,1,88.0,NEW GENERATION,1978,230000.0,78,3560 days
46816,2014-09-01,BUKIT MERAH,4 ROOM,13,CANTONMENT CL,4,90.0,MODEL A,2003,748000.0,88,9009 days
195266,2005-12-01,BUKIT BATOK,4 ROOM,288C,BT BATOK ST 25,0,93.0,MODEL A2,1998,260000.0,92,5813 days


In [29]:
# We can just go ahead and convert the data type to an integer for easier feeding in to the learning model!
transactions['tx_day'] = transactions['tx_day'].dt.days.astype(int)
transactions.sample(5)

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease,tx_day
235200,2007-05-01,WOODLANDS,4 ROOM,684A,WOODLANDS DR 73,1,104.0,PREMIUM APARTMENT,2000,278000.0,92,6329
45824,2014-08-01,JURONG WEST,5 ROOM,686C,JURONG WEST CTRL 1,1,111.0,PREMIUM APARTMENT,2000,545000.0,85,8978
6206,2012-05-01,SERANGOON,4 ROOM,232,SERANGOON AVE 3,0,84.0,SIMPLIFIED,1985,452000.0,72,8156
296813,2009-08-01,TAMPINES,3 ROOM,215,TAMPINES ST 23,1,67.0,NEW GENERATION,1985,240000.0,75,7152
146225,2004-04-01,ANG MO KIO,3 ROOM,444,ANG MO KIO AVE 10,1,82.0,NEW GENERATION,1979,205000.0,74,5204


## We're almost at the end of feature engineering, just the address to go. 
The plan is to combine the block + street name to obtain the full address, and with the full address, obtain a postal code. However there aren't any ready database for us to obtain postal codes from an address, so we're going to have to scrape it from the Singpost website.

In [30]:
import time
import requests
from bs4 import BeautifulSoup
#import required packages for scraping

In [33]:
transactions['full_address'] = transactions['block'] + ' ' + transactions['street_name']
all_addresses = transactions['full_address'].unique()
print('There are {} addresses'.format(len(all_addresses)))
transactions.sample(5)

There are 9279 addresses


,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease,tx_day,full_address
62997,1994-02-01,YISHUN,4 ROOM,439,YISHUN AVE 11,2,104.0,MODEL A,1988,173000.0,93,1492,439 YISHUN AVE 11
279326,1999-10-01,KALLANG/WHAMPOA,4 ROOM,121,MCNAIR RD,2,104.0,MODEL A,1987,335000.0,87,3560,121 MCNAIR RD
129488,2003-10-01,BEDOK,3 ROOM,30,NEW UPP CHANGI RD,2,65.0,IMPROVED,1977,170000.0,73,5021,30 NEW UPP CHANGI RD
189405,2005-09-01,TAMPINES,4 ROOM,293,TAMPINES ST 22,2,105.0,MODEL A,1984,234000.0,78,5722,293 TAMPINES ST 22
16003,2000-06-01,YISHUN,3 ROOM,746,YISHUN ST 72,0,67.0,NEW GENERATION,1984,162000.0,83,3804,746 YISHUN ST 72


Looks pretty good! We have 9279 addresses to scrape from the singpost website, should be the most hits they get in a day. HAHA.

In [34]:
# Here's the function I used to obtain the postal codes from the HDB addresses in this dataset.
def address_to_Pcode(building, address):
    '''
    Uses requests and beautifulsoup to post an address to singpost website to obtain the postal code of said address.
    '''
    url = "https://www.singpost.com/find-postal-code"
    headers = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.75 Safari/537.36'}
    data = {'building': building,
            'street_name': address,
            'op': 'Find',
           }
    r = requests.post(url, data = data, headers = headers)
    if r.status_code != 200:
        print('Request unsuccessful')
        return building+address
    else:
        soup = BeautifulSoup(r.text,'html.parser')
        datatable = soup.find(id="datatable-1")
        if datatable:
            postal_code = datatable.td.find_all('p')[1].get_text()
            return postal_code
        else:
            return 'Address not found.'

def get_postal_codes(file, all_addresses,n):
    '''
    file: dictionary containing obtained addresses and postal codes
    all_addresses: list of unique addresses/block numbers
    n: number of codes to retrieve per run
    returns dictionary of addresses & list of failed addresses in a tuple
    '''
    if type(file) != dict:
        raise TypeError('file should be a dictionary.')
    postal_code_dict = file
    failed_addresses = []
    count = 0
    for add in all_addresses:
        if count >= n:
            break
        if add not in postal_code_dict:
            block, address = add.split(maxsplit = 1)
            postal_code = address_to_Pcode(block, address)
            if len(postal_code)!=6:
                print('Address failed')
                postal_code_dict[add] = postal_code # adds 
                failed_addresses.append(add)
            else:
                print(add,'successful, adding to dictionary... count is:',count)
                postal_code_dict[add] = postal_code
                count += 1
            time.sleep(1) # be sure to let your function sleep for a bit, in case the server bans you!
        else: continue
    return postal_code_dict, failed_addresses

# I changed this to a markdown so I don't run this code again, but you can if you want to scrape the postal codes yourself.
```python
postal_code_dict, cur_failed_addresses = get_postal_codes(postal_code_dict,all_addresses,len(all_addresses))
failed_addresses.extend(cur_failed_addresses)
```
I saved the address/postalcode dictionary so I won't have to go through the hassle of scraping the website, just in case if I need it again.

In [36]:
def pickleRick(file,file_name):
    import pickle
    outfile = open(file_name,'wb')
    pickle.dump(file,outfile)
    outfile.close()

def unpickleRick(file_name):
    import pickle
    infile = open(file_name, 'rb')
    file = pickle.load(infile)
    infile.close()
    return file

#Don't mind the Rick & Morty references!

In [37]:
postal_code_dict = unpickleRick('postal_code.pickle')

In [39]:
# time to map our addresses to postal codes!
transactions['postalcode'] = transactions['full_address'].map(postal_code_dict)
transactions.sample(5)

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease,tx_day,full_address,postalcode
22867,2013-03-01,CHOA CHU KANG,3 ROOM,2,TECK WHYE AVE,0,76.0,MODEL A,1984,341000.0,70,8460,2 TECK WHYE AVE,680002
47939,2001-05-01,GEYLANG,5 ROOM,40,SIMS DR,2,124.0,IMPROVED,1990,365000.0,88,4138,40 SIMS DR,380040
19582,2017-12-01,JURONG WEST,3 ROOM,916,JURONG WEST ST 91,1,73.0,MODEL A,1988,270000.0,69,10196,916 JURONG WEST ST 91,640916
169169,1997-09-01,CLEMENTI,3 ROOM,304,CLEMENTI AVE 4,2,67.0,NEW GENERATION,1979,203000.0,81,2800,304 CLEMENTI AVE 4,120304
141485,1996-11-01,BISHAN,5 ROOM,179,BISHAN ST 13,4,121.0,IMPROVED,1987,600000.0,90,2496,179 BISHAN ST 13,570179


In [43]:
#clean up some of the mess due to my trying of different methods when creating my postalcode/address dictionary
pcs = transactions.postalcode.unique()
fails = []
for pc in pcs:
    try: int(pc)
    except: fails.append(pc)
faildict = {add:np.nan for add in fails}
transactions.replace({"postalcode":faildict},inplace=True)

In [44]:
transactions['postalcode'].isna().sum()

28851

Some missing data, possible reasons are that address no longer exists because of a change in road names, my mistake in encoding the address when trying out my webscraping etc. Luckily, we have a huge sample size, we can afford to lose these samples (~3%) without too much hurt! Until I have time to go fix the failed addresses one day!

In [45]:
transactions = transactions.dropna()
transactions.dtypes

month                  datetime64[ns]
town                           object
flat_type                      object
block                          object
street_name                    object
storey_range                    int64
floor_area_sqm                float64
flat_model                     object
lease_commence_date             int64
resale_price                  float64
remaining_lease                 int64
tx_day                          int64
full_address                   object
postalcode                    float64
dtype: object

In [46]:
transactions.isna().sum()

month                  0
town                   0
flat_type              0
block                  0
street_name            0
storey_range           0
floor_area_sqm         0
flat_model             0
lease_commence_date    0
resale_price           0
remaining_lease        0
tx_day                 0
full_address           0
postalcode             0
dtype: int64

## Here we have it, a clean dataset, cleaned and transformed where necessary, ready to be fed to your learner! This really was made way easier, because the dataset was very very clean to begin with, so kudos to our public service!

```python
pickleRick(transactions,'transactions.pickle')
#saving the transactions dataframe so you don't have to run the whole cleaning and transformation the next time you use it!
```